In [1]:
import nltk

nltk.download('stopwords')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

# Loadings

In [2]:
import warnings
warnings.filterwarnings("ignore")

import cross_validator as cross_validator
import datahandler as datahandler
from datamodel import DataModel
import outputmaker as outputmaker
import metrics as metrics

trial = datahandler.load_train('tsd_trial.csv', verbose=True)
train = datahandler.load_train('tsd_train.csv', verbose=True)

,spans,text
0,"[15, 16, 17, 18, 19, 27, 28, 29, 30, 31]",Because he's a moron and a bigot. It's not any...
1,"[29, 30, 31, 32, 33, 34]",How about we stop protecting idiots and let na...
2,"[166, 167, 168, 169, 170, 171]","If people were smart, they would Boycott th..."


,spans,text
0,"[8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,...",Another violent and aggressive immigrant killi...
1,"[33, 34, 35, 36, 37, 38, 39]","I am 56 years old, I am not your fucking junio..."
2,"[0, 1, 2, 3]","Damn, a whole family. Sad indeed."


In [3]:
datamodel = DataModel(model='crf', remove_stop_words=False)

output_maker = outputmaker.crf_output

evaluator = metrics

In [4]:
X_train, y_train, train_taboo_words, train_texts = cross_validator.load_data(train, [i for i in range(train.shape[0])],
                                                                             datamodel, logger=True)

X_test, y_test, test_taboo_words, test_texts = cross_validator.load_data(trial, [i for i in range(trial.shape[0])], 
                                                                   datamodel, logger=True)

100%|██████████| 690/690 [00:19<00:00, 36.28it/s]


# PreProcessing

In [5]:
from nltk.stem.porter import *
from tqdm import tqdm 

stemmer = PorterStemmer()

special_chars = ['O', '4', '͞', 'ü', '😞', '»', '”', '\x7f', '"', '7', '*', '8', ')',
      '😂', '💀', '{', '0', 'ʻ', '💨', '•', '#', '_', '😊', '😜', 'ó', '😅', 
      '¬', '☠', '🙄', '😉', '😆', '―','(', '.', '?', '😵', '💥', ':', '🆘',
      '$', '=', '+', ';', '🔥', '😁','`', 'ï', '2', 'ê', 'é', '3', '-', '🤥',
      '⚽', 'É', '️', '!', '5', '✭', '9', '😈', 'ˈ', '😬', '@', '«', '/', '▀', '’',
      '😡', '1', '%', ',', '&', '6', '\\', ']', "'", '[', '^', '}', '⚾', '\u200b']

F_cleaner = lambda x: ''.join([w for w in x if w not in special_chars ])

     
def cleaning(X):
    cleaned_x = []
    for words in tqdm(X):
        cleaned_words = []
        for word in words :
            w, pos, tag = word
            cleaned_w = F_cleaner(w)
            if len(cleaned_w) != 0:
                cleaned_words.append( (stemmer.stem(cleaned_w), pos, tag) )
        cleaned_x.append(cleaned_words)
    return cleaned_x


In [6]:
X_train = cleaning(X_train)
X_test = cleaning(X_test)

100%|██████████| 690/690 [00:00<00:00, 1164.31it/s]


In [7]:
X_train[0]

[('anoth', 'DT', 'normal'),
 ('violent', 'JJ', 'toxic'),
 ('and', 'CC', 'toxic'),
 ('aggress', 'JJ', 'toxic'),
 ('immigr', 'NN', 'toxic'),
 ('kill', 'VBG', 'normal'),
 ('a', 'DT', 'normal'),
 ('innoc', 'JJ', 'normal'),
 ('and', 'CC', 'normal'),
 ('intellig', 'JJ', 'normal'),
 ('US', 'NNP', 'normal'),
 ('citizen', 'NNP', 'normal'),
 ('sarcasm', 'NNP', 'normal')]

# Prepare for modeling

In [8]:
words = set([w for x in X_train + X_test for w, tag, label in x])
chars = set([char for x in X_train + X_test for w, tag, label in x for char in w])
tags = set([tag for x in X_train for w, pos, tag in x])

n_tags = len(tags)
n_chars = len(chars)
n_words = len(words)
n_tags = len(tags)
n_chars = len(chars)
n_words = len(words)

n_tags, n_chars, n_words

(2, 51, 13715)

In [9]:
print(chars)

{'w', 'v', 'u', 'S', 'p', 'x', 'C', 'T', 'H', 'G', 'Q', 'V', 'X', 'k', 't', 'i', 'N', 'M', 'f', 'L', 'e', 'y', 'R', 'U', 'Y', 'D', 'K', 'g', 'o', 'J', 'z', 'm', 'b', 'A', 'P', 'a', 's', 'd', 'n', 'B', 'h', 'E', 'c', 'q', 'I', 'j', 'F', 'Z', 'l', 'r', 'W'}


In [10]:
max([len(x) for x in X_train+X_test]), max([len(w) for x in X_train+X_test for w, tag, label in x])

(191, 149)

In [11]:
max_len = 191
max_len_char = 149

In [12]:
word2idx = {w: i + 2 for i, w in enumerate(words)}
word2idx["UNK"] = 1
word2idx["PAD"] = 0

idx2word = {i: w for w, i in word2idx.items()}

tag2idx = {t: i + 1 for i, t in enumerate(tags)}
tag2idx["PAD"] = 0
idx2tag = {i: w for w, i in tag2idx.items()}

In [13]:
print(word2idx["fuck"])
print(tag2idx["normal"])

10090
1


In [14]:
from keras.preprocessing.sequence import pad_sequences

X_word = [[word2idx[w[0]] for w in s] for s in X_train]

In [15]:
X_word = pad_sequences(maxlen=max_len, sequences=X_word, value=word2idx["PAD"], padding='post', truncating='post')

In [16]:
char2idx = {c: i + 2 for i, c in enumerate(chars)}
char2idx["UNK"] = 1
char2idx["PAD"] = 0

In [17]:
import numpy as np

X_char = []
for sentence in X_train:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char.append(np.array(sent_seq))

In [18]:
X_char[0]

array([[37, 40, 30, ...,  0,  0,  0],
       [ 3, 17, 30, ...,  0,  0,  0],
       [37, 40, 39, ...,  0,  0,  0],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [19]:
y = [[tag2idx[w[2]] for w in s] for s in X_train]

len(y)

7939

In [20]:
y = pad_sequences(maxlen=max_len, sequences=y, value=tag2idx["PAD"], padding='post', truncating='post')

# Train Test split

In [21]:
from sklearn.model_selection import train_test_split

X_word_tr, y_tr = X_word, y
X_char_tr = X_char

#X_word_tr, X_word_te, y_tr, y_te = train_test_split(X_word, y, test_size=0.1, random_state=2018)
#X_char_tr, X_char_te, _, _ = train_test_split(X_char, y, test_size=0.1, random_state=2018)

In [ ]:
#X_word_te.shape

# word-char based Bi-LSTM 

In [22]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Conv1D
from keras.layers import Bidirectional, concatenate, SpatialDropout1D, GlobalMaxPooling1D

In [27]:
# input and embedding for words
word_in = Input(shape=(max_len,))
emb_word = Embedding(input_dim=n_words + 2, output_dim=50,
                     input_length=max_len, mask_zero=True)(word_in)

# input and embeddings for characters
char_in = Input(shape=(max_len, max_len_char,))
emb_char = TimeDistributed(Embedding(input_dim=n_chars + 2, output_dim=20,
                           input_length=max_len_char, mask_zero=True))(char_in)
# character LSTM to get word encodings by characters
char_enc = TimeDistributed(LSTM(units=50, return_sequences=False,
                                recurrent_dropout=0.5))(emb_char)

# main LSTM
x = concatenate([emb_word, char_enc])
x = SpatialDropout1D(0.3)(x)
main_lstm = Bidirectional(LSTM(units=50, return_sequences=True,
                               recurrent_dropout=0.4))(x)
out = TimeDistributed(Dense(n_tags + 1, activation="sigmoid"))(main_lstm)

model = Model([word_in, char_in], out)

In [28]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["acc"])

In [29]:
model.summary()

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 191, 149)]   0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 191)]        0                                            
__________________________________________________________________________________________________
time_distributed_3 (TimeDistrib (None, 191, 149, 20) 1060        input_4[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 191, 50)      685850      input_3[0][0]                    
_______________________________________________________________________________________

In [ ]:
history = model.fit([X_word_tr,
                    np.array(X_char_tr).reshape((len(X_char_tr), max_len, max_len_char))],
                    np.array(y_tr).reshape(len(y_tr), max_len, 1),
                    batch_size=32, epochs=10, validation_split=0.1, verbose=1)

Epoch 1/10
 64/224 [=======>......................] - ETA: 11:32 - loss: 0.0918 - acc: 0.8997

In [ ]:
import pandas as pd 
hist = pd.DataFrame(history.history)

In [ ]:
import matplotlib.pyplot as plt
plt.style.use("ggplot")
plt.figure(figsize=(12,12))
plt.plot(hist["acc"])
plt.plot(hist["val_acc"])
plt.show()

In [ ]:
y_pred = model.predict([X_word_te,
                        np.array(X_char_te).reshape((len(X_char_te),
                                                     max_len, max_len_char))])
y_pred[0][0]

array([1.1361879e-04, 9.6667194e-01, 1.9100308e-04], dtype=float32)

In [ ]:
np.array(X_char_te).reshape((len(X_char_te),max_len, max_len_char)).shape

(794, 157, 198)

In [ ]:
#y_pred.shape[0]794
y_prediction = []
y_truth = []

In [ ]:
print("{:15}||{:5}||{}".format("Word", "True", "Pred"))
print(30 * "=")

for i in range(y_pred.shape[0]):
    #i = 314
    p = np.argmax(y_pred[i], axis=-1)
    
    for w, t, pred in zip(X_word_te[i], y_te[i], p):
        if w != 0:
            y_prediction.append(idx2tag[pred])
            y_truth.append(idx2tag[t])
            #if idx2tag[t] == 'toxic' or idx2tag[pred] == 'toxic':
            #    print("{} {:15}: {:5} {}".format(i, idx2word[w], idx2tag[t], idx2tag[pred]))


Word           ||True ||Pred


In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print(classification_report(y_truth, y_prediction))

              precision    recall  f1-score   support

      normal       0.93      0.96      0.95     12786
       toxic       0.59      0.45      0.51      1652

    accuracy                           0.90     14438
   macro avg       0.76      0.70      0.73     14438
weighted avg       0.89      0.90      0.90     14438



# Test

In [ ]:
X_test[0]

[('moron', 'NN', 'toxic'),
 ('bigot', 'NN', 'toxic'),
 ('complicated', 'JJ', 'normal')]

In [ ]:
X_word_te.shape

(794, 157)

In [ ]:
X_word_test = []
no = []
for s in X_test:
    x_s = []
    for w in s:
        try:
            x_s.append(word2idx[w[0]])
        except:
            no.append(w)
            
    X_word_test.append(x_s)
X_word_test = [[word2idx[w[0]] if w[0] in word2idx else word2idx["UNK"] for w in s] for s in X_test]

In [ ]:
len(no), len(X_word_test)

(0, 690)

In [ ]:

X_word_test = pad_sequences(maxlen=max_len, sequences=X_word_test, 
                            value=word2idx["PAD"], padding='post', truncating='post')

X_char_test = []
for sentence in X_test:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                word_seq.append(char2idx.get(sentence[i][0][j]))
            except:
                word_seq.append(char2idx.get("PAD"))
        sent_seq.append(word_seq)
    X_char_test.append(np.array(sent_seq).astype(float))

In [ ]:
y_pred_test = model.predict([X_word_test, np.array(X_char_test).reshape((len(X_char_test),max_len, max_len_char))])

In [ ]:
y_test_preds = []

In [ ]:
for i in range(y_pred_test.shape[0]):

    p = np.argmax(y_pred_test[i], axis=-1)
    preds = []
    for w, pred in zip(X_word_test[i], p):
        if w != 0:
            preds.append(idx2tag[pred])
    y_test_preds.append(preds)


In [ ]:
y_test_preds[0]

['toxic', 'toxic', 'normal']

In [ ]:
len(y_test_preds)

690

In [ ]:
predictions = output_maker(X_test, test_texts, y_test_preds)

f1, p, r = evaluator.evaluate(gold = y_test, predictions = predictions)

print("F1:{}, \t P:{}, \t R:{}".format(f1, p, r))

F1:0.5778733240805811, 	 P:0.6616228532625453, 	 R:0.603260666794722
